# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 4 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/AveragePooled/split_features_3s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[-0.01764554, -0.97810674, -0.97999614, ..., -0.82831436,
         -0.86151993, -0.9852102 ],
        [-0.0176447 , -0.97808164, -0.9799673 , ..., -0.82414514,
         -0.85394555, -0.982918  ],
        [-0.01764692, -0.97814673, -0.9800371 , ..., -0.81990373,
         -0.8500417 , -0.98001206],
        ...,
        [-0.01758811, -0.57715386, -0.5305197 , ..., -0.86590964,
         -0.8951364 , -0.9996821 ],
        [-0.01772401, -0.58169186, -0.5389059 , ..., -0.8691575 ,
         -0.89791137, -0.99996406],
        [-0.01760773, -0.57361674, -0.5377899 , ..., -0.8711317 ,
         -0.9001475 , -0.99997455]], dtype=float32),
 'mfcc': array([[-1.        , -0.07280675, -0.12869048, ...,  0.02178697,
          0.02212632,  0.04228346],
        [-1.        , -0.08856594, -0.15130392, ...,  0.02659525,
          0.01738   ,  0.04704501],
        [-1.        , -0.10337539, -0.15928885, ...,  0.02455455,
          0.01868342,  0.05395849],
        ...,
        [-0.9

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(13725,)

array([ 7, 12,  4, 17,  3,  9, 14,  3,  0,  0, 10,  2, 12,  3,  1],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(13725, 128)

array([-0.0581256 , -0.63822806, -0.6389076 , -0.4953763 , -0.36455613,
       -0.47280535, -0.6928881 , -0.7405781 , -0.7677548 , -0.7374399 ,
       -0.75045246, -0.77124953, -0.7853015 , -0.7958842 , -0.7959697 ,
       -0.81874114, -0.84561497, -0.85084915, -0.8656642 , -0.85950583,
       -0.8056054 , -0.82864517, -0.8715404 , -0.8579188 , -0.87162524,
       -0.8241646 , -0.80134803, -0.8806075 , -0.8931194 , -0.8950114 ,
       -0.89938545, -0.8664572 , -0.90240514, -0.9203411 , -0.9429153 ,
       -0.9389389 , -0.94072586, -0.94264907, -0.95205927, -0.9475349 ,
       -0.9499955 , -0.9482352 , -0.96123224, -0.96147287, -0.9631074 ,
       -0.9561097 , -0.9551953 , -0.94871837, -0.95351607, -0.94429374,
       -0.94507396, -0.93894887, -0.9448011 , -0.94699717, -0.9460722 ,
       -0.94137007, -0.9353987 , -0.9335354 , -0.92789626, -0.92009485,
       -0.9186622 , -0.9323982 , -0.93251014, -0.9347964 , -0.93165   ,
       -0.9263686 , -0.9260132 , -0.9232989 , -0.9198529 , -0.91

'mfcc'

(13725, 20)

...

(13725, 84)

array([-54.3924  , -55.247612, -54.019176, -55.412117, -55.176765,
       -51.641773, -51.608307, -56.090916, -57.880806, -58.957306,
       -57.609154, -57.304836, -56.412144, -55.77131 , -57.49621 ,
       -56.045795, -55.64338 , -56.29201 , -54.53955 , -55.223743,
       -56.586563, -55.49619 , -52.550323, -33.252346, -29.488544,
       -36.211555, -52.73948 , -57.32028 , -59.76588 , -60.917732,
       -62.20523 , -63.252007, -65.78648 , -67.615456, -67.43155 ,
       -65.66191 , -63.76412 , -64.753426, -67.14033 , -66.963356,
       -67.89089 , -69.045784, -68.605064, -69.198074, -70.61911 ,
       -72.82115 , -73.32479 , -74.19051 , -72.36656 , -70.252655,
       -73.62735 , -74.25865 , -72.172646, -70.6866  , -75.13241 ,
       -75.91342 , -75.68041 , -77.47689 , -78.32974 , -78.5558  ,
       -78.68926 , -78.85076 , -78.95114 , -78.8215  , -78.64342 ,
       -78.462425, -78.510155, -78.526566, -78.30219 , -78.121315,
       -77.88373 , -78.19443 , -77.83473 , -77.17658 , -76.501

'id'

(13725,)

'XC214099.mp3'

In [13]:
display(val_y.shape)
display(val_y[:15])

(4231,)

array([12, 18, 12,  1, 17, 16, 19, 16, 17,  8, 11,  4,  1,  9, 10],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(4231, 128)

array([-0.01764328, -0.9565721 , -0.93247706, -0.88549644, -0.81249005,
       -0.77785146, -0.76034915, -0.7679908 , -0.7814716 , -0.7643195 ,
       -0.7633203 , -0.76356155, -0.7534367 , -0.7411649 , -0.7278031 ,
       -0.7379763 , -0.74492484, -0.74983066, -0.76243466, -0.76931804,
       -0.78249747, -0.8023202 , -0.8106345 , -0.82639325, -0.8330959 ,
       -0.84451175, -0.849241  , -0.8568743 , -0.86505175, -0.88638127,
       -0.89514667, -0.90498   , -0.9097053 , -0.9165155 , -0.917027  ,
       -0.920565  , -0.9250685 , -0.92276466, -0.9206286 , -0.9269251 ,
       -0.934101  , -0.9314349 , -0.94306   , -0.9392371 , -0.9454409 ,
       -0.9512764 , -0.9590243 , -0.9694562 , -0.97636455, -0.9819564 ,
       -0.98757833, -0.9883887 , -0.99239784, -0.9925326 , -0.9923378 ,
       -0.9928755 , -0.9921703 , -0.98805094, -0.9837086 , -0.98562473,
       -0.9882742 , -0.9882152 , -0.98868024, -0.9876179 , -0.98632133,
       -0.98952055, -0.9845733 , -0.9795667 , -0.98120886, -0.98

'mfcc'

(4231, 20)

...

(4231, 84)

array([-58.28224 , -58.30469 , -60.03862 , -61.265427, -59.94281 ,
       -59.965263, -60.767155, -60.456078, -59.14285 , -58.579327,
       -56.73909 , -57.452988, -59.510807, -58.565063, -58.85057 ,
       -58.601093, -60.21403 , -58.776463, -57.312927, -58.795265,
       -56.85332 , -55.202473, -54.4058  , -53.438473, -52.28848 ,
       -48.160553, -48.251144, -48.59912 , -47.294586, -46.99907 ,
       -46.827312, -46.508904, -48.728092, -48.913437, -48.83963 ,
       -48.400917, -47.28493 , -47.166725, -47.661697, -47.72212 ,
       -47.44012 , -46.3531  , -45.144695, -45.080162, -45.847664,
       -46.067047, -46.776585, -47.623924, -48.124687, -50.001648,
       -51.211334, -52.777264, -53.999992, -55.038715, -55.989574,
       -57.921734, -59.345406, -59.930416, -60.376286, -60.95862 ,
       -61.72118 , -62.10022 , -62.469154, -63.866455, -66.58549 ,
       -67.87702 , -69.35963 , -70.02588 , -69.020294, -68.64077 ,
       -68.66529 , -68.9072  , -69.106926, -68.98829 , -68.596

'id'

(4231,)

'XC383514.mp3'

# **Random Forest Model**

Saving evaluation results

In [15]:
def evaluate_model(rf, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = rf.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = rf.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [16]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [17]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(13725, 244)

In [18]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(4231, 244)

Fit the model with training data

In [19]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [20]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.28574805010635784


In [21]:
val_scores['all_features'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.30      0.49      0.37       260
           1       0.41      0.55      0.47       424
           2       0.21      0.38      0.27       200
           3       0.19      0.32      0.24       318
           4       0.04      0.05      0.05        73
           5       0.00      0.00      0.00        99
           6       0.06      0.01      0.02        86
           7       0.51      0.39      0.45       224
           8       0.13      0.54      0.21        50
           9       0.15      0.32      0.21        71
          10       0.34      0.22      0.27       146
          11       0.29      0.01      0.02       549
          12       0.24      0.18      0.21       415
          13       0.31      0.09      0.14       160
          14       0.01      0.02      0.01        46
          15       0.25      0.34      0.29       255
          16       0.89      0.86      0.87   

### **With MFCCs only**

In [22]:
training_features = train_features['mfcc']

training_features.shape

(13725, 20)

In [23]:
validation_features = val_features['mfcc']

validation_features.shape

(4231, 20)

In [24]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.31387378870243443


In [26]:
val_scores['mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.30      0.71      0.42       260
           1       0.38      0.54      0.44       424
           2       0.21      0.35      0.27       200
           3       0.60      0.77      0.67       318
           4       0.03      0.01      0.02        73
           5       0.00      0.00      0.00        99
           6       0.08      0.01      0.02        86
           7       0.31      0.52      0.38       224
           8       0.30      0.48      0.37        50
           9       0.20      0.44      0.28        71
          10       0.29      0.21      0.24       146
          11       0.55      0.01      0.02       549
          12       0.31      0.23      0.27       415
          13       0.52      0.39      0.44       160
          14       0.05      0.09      0.07        46
          15       0.15      0.35      0.21       255
          16       0.56      0.12      0.20   

### **With Mel-Spectrogram Only**

In [27]:
training_features = train_features['melspectrogram']

training_features.shape

(13725, 128)

In [28]:
validation_features = val_features['melspectrogram']

validation_features.shape

(4231, 128)

In [29]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [30]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2590404159773103


In [31]:
val_scores['melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.28      0.50      0.36       260
           1       0.36      0.56      0.44       424
           2       0.23      0.38      0.29       200
           3       0.21      0.29      0.25       318
           4       0.02      0.03      0.02        73
           5       0.02      0.01      0.01        99
           6       0.05      0.01      0.02        86
           7       0.36      0.28      0.31       224
           8       0.12      0.54      0.20        50
           9       0.16      0.27      0.20        71
          10       0.31      0.19      0.24       146
          11       0.19      0.01      0.01       549
          12       0.20      0.17      0.18       415
          13       0.34      0.11      0.17       160
          14       0.01      0.02      0.01        46
          15       0.26      0.38      0.31       255
          16       0.72      0.47      0.57   

## **With Chroma Only**

In [32]:
training_features = train_features['chroma']

training_features.shape

(13725, 12)

In [33]:
validation_features = val_features['chroma']

validation_features.shape

(4231, 12)

In [34]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [35]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 0.9998542805100182
Validation accuracy: 0.21933349090049634


In [36]:
val_scores['chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.45      0.19       260
           1       0.32      0.40      0.35       424
           2       0.11      0.41      0.17       200
           3       0.74      0.60      0.66       318
           4       0.00      0.00      0.00        73
           5       0.50      0.15      0.23        99
           6       0.33      0.01      0.02        86
           7       0.16      0.21      0.18       224
           8       0.04      0.02      0.03        50
           9       0.17      0.15      0.16        71
          10       0.05      0.03      0.04       146
          11       0.16      0.01      0.01       549
          12       0.27      0.07      0.11       415
          13       0.31      0.11      0.16       160
          14       0.06      0.09      0.07        46
          15       0.24      0.45      0.31       255
          16       0.29      0.20      0.24   

## **With CQT Only**

In [37]:
training_features = train_features['cqt']

training_features.shape

(13725, 84)

In [38]:
validation_features = val_features['cqt']

validation_features.shape

(4231, 84)

In [39]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [40]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2176790356889624


In [41]:
val_scores['cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.18      0.13       260
           1       0.34      0.50      0.41       424
           2       0.00      0.01      0.00       200
           3       0.23      0.29      0.26       318
           4       0.00      0.00      0.00        73
           5       0.00      0.00      0.00        99
           6       0.07      0.02      0.03        86
           7       0.39      0.38      0.39       224
           8       0.24      0.50      0.33        50
           9       0.10      0.23      0.13        71
          10       0.30      0.19      0.23       146
          11       0.00      0.00      0.00       549
          12       0.20      0.17      0.18       415
          13       0.44      0.14      0.21       160
          14       0.02      0.07      0.03        46
          15       0.15      0.18      0.16       255
          16       0.67      0.79      0.73   

## **With MFCCs and Mel-Spectrogram**

In [42]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(13725, 148)

In [43]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(4231, 148)

In [44]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2666036398014654


In [46]:
val_scores['mfcc_melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.31      0.53      0.39       260
           1       0.31      0.57      0.40       424
           2       0.23      0.39      0.29       200
           3       0.35      0.47      0.40       318
           4       0.03      0.04      0.04        73
           5       0.02      0.01      0.01        99
           6       0.00      0.00      0.00        86
           7       0.35      0.28      0.31       224
           8       0.12      0.52      0.19        50
           9       0.12      0.28      0.17        71
          10       0.27      0.17      0.21       146
          11       0.18      0.01      0.01       549
          12       0.23      0.20      0.21       415
          13       0.42      0.14      0.21       160
          14       0.02      0.07      0.03        46
          15       0.26      0.37      0.31       255
          16       0.55      0.20      0.30   

## **With MFCCs and Chroma**

In [47]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(13725, 32)

In [48]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(4231, 32)

In [49]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [50]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3415268258095013


In [51]:
val_scores['mfcc_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.29      0.77      0.42       260
           1       0.37      0.54      0.44       424
           2       0.21      0.40      0.28       200
           3       0.60      0.76      0.67       318
           4       0.04      0.01      0.02        73
           5       0.00      0.00      0.00        99
           6       0.10      0.01      0.02        86
           7       0.32      0.46      0.38       224
           8       0.28      0.50      0.36        50
           9       0.25      0.39      0.31        71
          10       0.46      0.26      0.33       146
          11       0.38      0.01      0.02       549
          12       0.43      0.29      0.35       415
          13       0.56      0.37      0.45       160
          14       0.00      0.00      0.00        46
          15       0.18      0.40      0.25       255
          16       0.74      0.39      0.51   

## **With MFCCs and CQT**

In [52]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(13725, 104)

In [53]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(4231, 104)

In [54]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [55]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2703852517135429


In [56]:
val_scores['mfcc_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.21      0.36      0.27       260
           1       0.37      0.52      0.43       424
           2       0.10      0.20      0.13       200
           3       0.24      0.35      0.29       318
           4       0.05      0.03      0.03        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.43      0.50      0.46       224
           8       0.16      0.48      0.24        50
           9       0.13      0.30      0.19        71
          10       0.13      0.08      0.10       146
          11       0.12      0.00      0.00       549
          12       0.34      0.31      0.32       415
          13       0.62      0.15      0.24       160
          14       0.01      0.04      0.02        46
          15       0.18      0.22      0.19       255
          16       0.85      0.84      0.84   

## **With Chroma and CQT**

In [57]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(13725, 96)

In [58]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(4231, 96)

In [59]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [60]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2216969983455448


In [61]:
val_scores['chroma_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.11      0.19      0.14       260
           1       0.34      0.50      0.40       424
           2       0.06      0.14      0.09       200
           3       0.24      0.30      0.26       318
           4       0.00      0.00      0.00        73
           5       0.00      0.00      0.00        99
           6       0.07      0.03      0.05        86
           7       0.35      0.35      0.35       224
           8       0.22      0.48      0.30        50
           9       0.09      0.23      0.12        71
          10       0.20      0.11      0.14       146
          11       0.13      0.00      0.01       549
          12       0.17      0.13      0.15       415
          13       0.46      0.15      0.23       160
          14       0.01      0.04      0.02        46
          15       0.15      0.20      0.17       255
          16       0.72      0.82      0.77   

## **With Mel-Spectrogram and Chroma**

In [62]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(13725, 140)

In [63]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(4231, 140)

In [64]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2703852517135429


In [66]:
val_scores['melspectrogram_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.28      0.50      0.35       260
           1       0.36      0.55      0.43       424
           2       0.25      0.39      0.30       200
           3       0.26      0.34      0.30       318
           4       0.02      0.03      0.02        73
           5       0.00      0.00      0.00        99
           6       0.07      0.01      0.02        86
           7       0.41      0.28      0.33       224
           8       0.11      0.52      0.19        50
           9       0.17      0.30      0.21        71
          10       0.26      0.16      0.20       146
          11       0.21      0.01      0.02       549
          12       0.21      0.16      0.18       415
          13       0.32      0.09      0.14       160
          14       0.01      0.04      0.02        46
          15       0.26      0.42      0.32       255
          16       0.71      0.53      0.61   

## **With Mel-Spectrogram and CQT**

In [67]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(13725, 212)

In [68]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(4231, 212)

In [69]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [70]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.27274875915859137


In [71]:
val_scores['melspectrogram_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.23      0.32      0.27       260
           1       0.40      0.55      0.46       424
           2       0.20      0.36      0.26       200
           3       0.17      0.29      0.21       318
           4       0.04      0.05      0.05        73
           5       0.00      0.00      0.00        99
           6       0.06      0.01      0.02        86
           7       0.61      0.38      0.46       224
           8       0.12      0.54      0.20        50
           9       0.13      0.31      0.18        71
          10       0.40      0.21      0.27       146
          11       0.29      0.01      0.01       549
          12       0.25      0.20      0.22       415
          13       0.41      0.11      0.18       160
          14       0.02      0.07      0.03        46
          15       0.21      0.29      0.25       255
          16       0.84      0.86      0.85   

## **With Mel-Spectrogram, MFCCs, CQT**

In [72]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(13725, 232)

In [73]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(4231, 232)

In [74]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [75]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2767667218151737


In [76]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.24      0.40      0.30       260
           1       0.33      0.56      0.41       424
           2       0.20      0.35      0.26       200
           3       0.23      0.32      0.27       318
           4       0.04      0.05      0.05        73
           5       0.00      0.00      0.00        99
           6       0.08      0.01      0.02        86
           7       0.49      0.34      0.40       224
           8       0.12      0.54      0.19        50
           9       0.18      0.30      0.23        71
          10       0.34      0.19      0.25       146
          11       0.31      0.01      0.01       549
          12       0.24      0.19      0.21       415
          13       0.37      0.09      0.14       160
          14       0.02      0.07      0.03        46
          15       0.22      0.31      0.26       255
          16       0.87      0.87      0.87   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [77]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(13725, 160)

In [78]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(4231, 160)

In [79]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [80]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2739305128811156


In [81]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.30      0.53      0.39       260
           1       0.34      0.57      0.42       424
           2       0.27      0.43      0.33       200
           3       0.35      0.54      0.43       318
           4       0.03      0.04      0.03        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.37      0.32      0.35       224
           8       0.13      0.54      0.21        50
           9       0.10      0.30      0.15        71
          10       0.29      0.18      0.23       146
          11       0.18      0.01      0.01       549
          12       0.26      0.18      0.21       415
          13       0.33      0.13      0.19       160
          14       0.02      0.07      0.03        46
          15       0.26      0.37      0.30       255
          16       0.52      0.22      0.31   

## Review results from all models

In [82]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.286
3,chroma,1.0,0.219
8,chroma_cqt,1.0,0.222
4,cqt,1.0,0.218
2,melspectrogram,1.0,0.259
9,melspectrogram_chroma,1.0,0.270
12,melspectrogram_chroma_mfcc,1.0,0.274
10,melspectrogram_cqt,1.0,0.273
11,melspectrogram_cqt_mfcc,1.0,0.277
1,mfcc,1.0,0.314


In [83]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(2)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features      0.26       0.76
3                    chroma      0.20       0.71
8                chroma_cqt      0.20       0.71
4                       cqt      0.20       0.71
2            melspectrogram      0.24       0.73
9     melspectrogram_chroma      0.25       0.75
10       melspectrogram_cqt      0.25       0.73
11  melspectrogram_cqt_mfcc      0.25       0.74
1                      mfcc      0.28       0.80
6               mfcc_chroma      0.31       0.84
7                  mfcc_cqt      0.25       0.74
5       mfcc_melspectrogram      0.24       0.74
